# Sequential Data transformations

In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from Modules import *
sns.set()
%matplotlib inline

In [3]:
df, y = read_data('default_of_credit_card_clients.xls')
df = proc_cat_df(df)


df.head()

LIMIT_BAL  AGE  PAY_1  PAY_2  PAY_3  PAY_4  PAY_5  PAY_6  BILL_AMT1  \
1      20000   24      2      2     -1     -1     -2     -2       3913   
2     120000   26     -1      2      0      0      0      2       2682   
3      90000   34      0      0      0      0      0      0      29239   
4      50000   37      0      0      0      0      0      0      46990   
5      50000   57     -1      0     -1      0      0      0       8617   

   BILL_AMT2          ...           PAY_AMT5  PAY_AMT6  Y  SEX_Female  \
1       3102          ...                  0         0  1           1   
2       1725          ...                  0      2000  1           1   
3      14027          ...               1000      5000  0           1   
4      48233          ...               1069      1000  0           1   
5       5670          ...                689       679  0           0   

   SEX_Male  EDUCATION_Graduate School  EDUCATION_Other  EDUCATION_University  \
1         0                          0                0                     1   
2         0                          0                0                     1   
3         0                          0                0                     1   
4         0                          0                0                     1   
5         1                          0                0                     1   

   MARRIAGE_Married  MARRIAGE_Non-married  
1                 1                     0  
2                 0                     1  
3                 0                     1  
4                 1                     0  
5                 1                     0  

[5 rows x 28 columns]

In [4]:
df_copy =  df.iloc[:, 2:20].copy()
df_copy.head(10)

PAY_1  PAY_2  PAY_3  PAY_4  PAY_5  PAY_6  BILL_AMT1  BILL_AMT2  BILL_AMT3  \
1       2      2     -1     -1     -2     -2       3913       3102        689   
2      -1      2      0      0      0      2       2682       1725       2682   
3       0      0      0      0      0      0      29239      14027      13559   
4       0      0      0      0      0      0      46990      48233      49291   
5      -1      0     -1      0      0      0       8617       5670      35835   
6       0      0      0      0      0      0      64400      57069      57608   
7       0      0      0      0      0      0     367965     412023     445007   
8       0     -1     -1      0      0     -1      11876        380        601   
9       0      0      2      0      0      0      11285      14096      12108   
10     -2     -2     -2     -2     -1     -1          0          0          0   

    BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  PAY_AMT4  \
1           0          0          0         0       689         0         0   
2        3272       3455       3261         0      1000      1000      1000   
3       14331      14948      15549      1518      1500      1000      1000   
4       28314      28959      29547      2000      2019      1200      1100   
5       20940      19146      19131      2000     36681     10000      9000   
6       19394      19619      20024      2500      1815       657      1000   
7      542653     483003     473944     55000     40000     38000     20239   
8         221       -159        567       380       601         0       581   
9       12211      11793       3719      3329         0       432      1000   
10          0      13007      13912         0         0         0     13007   

    PAY_AMT5  PAY_AMT6  
1          0         0  
2          0      2000  
3       1000      5000  
4       1069      1000  
5        689       679  
6       1000       800  
7      13750     13770  
8       1687      1542  
9       1000      1000  
10      1122         0

## TO DO:

need to make a baseline classifier; probably just a basic xgboost

## Baseline Classifier

This cell below is the base evaluation of the above dataframe, and will be the same for Martin's new data that he has created

In [23]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import sklearn.metrics
from imblearn.over_sampling import RandomOverSampler

X, y = df.drop(columns = ['Y']), df['Y']

data_dmatrix = xgb.DMatrix(data=X,label=y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2019)

oversampler = RandomOverSampler(random_state = 2019)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

print(X_train_oversampled.shape())

xg_clf = xgb.XGBClassifier()

#xg_clf.fit(X_train, y_train)


ImportError: cannot import name 'pairwise_distances_chunked'

In [19]:
#create some hyper-parameters, choose 'auc' as scoring metric
#cross validate the metrics

params = {"objective":'binary:logistic','colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

#put results as a pandas dataframe
#will track auc and mean avg precision (map), as a proxy for f1 score
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics=["auc", 'map'],as_pandas=True, seed=2019)



In [21]:
cv_results.tail()

train-auc-mean  train-auc-std  train-map-mean  train-map-std  \
45        0.800601       0.002512        0.586951       0.003669   
46        0.801045       0.002473        0.587952       0.003380   
47        0.801787       0.002408        0.588623       0.003567   
48        0.802321       0.002221        0.589580       0.004015   
49        0.802783       0.002238        0.590168       0.003900   

    test-auc-mean  test-auc-std  test-map-mean  test-map-std  
45       0.780800      0.005327       0.555268      0.007818  
46       0.780840      0.005338       0.555640      0.008056  
47       0.781209      0.005494       0.555646      0.008111  
48       0.781424      0.005605       0.555617      0.007705  
49       0.781522      0.005545       0.555839      0.007934